In [1]:
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.datasets import imdb
from keras.utils import pad_sequences

2024-01-02 23:52:48.386706: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-02 23:52:48.435381: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 23:52:48.435417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 23:52:48.436526: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 23:52:48.443847: I tensorflow/core/platform/cpu_feature_guar

In [2]:
class TransformerBlock(Layer):
  def __init__(self, embed_dim, num_head, ff_dim, rate  = 0.1):

    super().__init__()
    self.att = MultiHeadAttention(num_heads = num_head , key_dim = embed_dim)

    self.ffn = Sequential(
        [Dense(ff_dim, activation = 'relu'), Dense(embed_dim)],
    )

    self.layernorm1 = LayerNormalization(epsilon = 1e-6)
    self.layernorm2 = LayerNormalization(epsilon = 1e-6)

    self.dropout1 = Dropout(rate)
    self.dropout2 = Dropout(rate)


  def call(self, inputs, training):
    attn_output = self.att(inputs, inputs)

    attn_output = self.dropout1(attn_output, training = training)

    out1 = self.layernorm1(inputs + attn_output)

    ffn_output = self.ffn(out1)

    ffn_output = self.dropout2(ffn_output, training = training)

    return self.layernorm2(out1 + ffn_output)

# Implementing Embedding Layer

In [3]:
class TokenAndPositionEmbedding(Layer):

  def __init__(self, maxlen, vocab_size, embed_dim):
    super().__init__()
    self.token_emb = Embedding(input_dim= vocab_size, output_dim=embed_dim)
    self.pos_emb = Embedding(input_dim= maxlen, output_dim=embed_dim)


  def call(self, x):
    maxlen = tf.shape(x)[-1]
    positions = tf.range(start = 0, limit = maxlen, delta = 1)
    postions = self.pos_emb(positions)
    x = self.token_emb(x)
    return x + postions

In [4]:
vocab_size = 20000 # Only consider the top 20k words
maxlen = 200
(x_train, y_train), (x_val, y_val) = imdb.load_data(num_words = vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = pad_sequences(x_train, maxlen = maxlen)
x_val = pad_sequences(x_val, maxlen = maxlen)

17464789/17464789 [==============================] - 3s 0us/step
25000 Training sequences
25000 Validation sequences


In [5]:
x_train[0]

array([    5,    25,   100,    43,   838,   112,    50,   670,     2,
           9,    35,   480,   284,     5,   150,     4,   172,   112,
         167,     2,   336,   385,    39,     4,   172,  4536,  1111,
          17,   546,    38,    13,   447,     4,   192,    50,    16,
           6,   147,  2025,    19,    14,    22,     4,  1920,  4613,
         469,     4,    22,    71,    87,    12,    16,    43,   530,
          38,    76,    15,    13,  1247,     4,    22,    17,   515,
          17,    12,    16,   626,    18, 19193,     5,    62,   386,
          12,     8,   316,     8,   106,     5,     4,  2223,  5244,
          16,   480,    66,  3785,    33,     4,   130,    12,    16,
          38,   619,     5,    25,   124,    51,    36,   135,    48,
          25,  1415,    33,     6,    22,    12,   215,    28,    77,
          52,     5,    14,   407,    16,    82, 10311,     8,     4,
         107,   117,  5952,    15,   256,     4,     2,     7,  3766,
           5,   723,

In [6]:
x_train.shape

(25000, 200)

In [7]:
embed_dim = 32
num_heads = 2
ff_dim = 32

inputs = Input(shape = (maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation = 'relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(2, activation = 'softmax')(x)

model = Model(inputs = inputs, outputs = outputs)

In [8]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

history = model.fit(x_train, y_train, batch_size = 32, epochs = 10, validation_data = (x_val, y_val))

Epoch 1/10
782/782 [==============================] - 37s 44ms/step - loss: 0.3798 - accuracy: 0.8209 - val_loss: 0.2878 - val_accuracy: 0.8784
Epoch 2/10
782/782 [==============================] - 35s 45ms/step - loss: 0.1930 - accuracy: 0.9256 - val_loss: 0.3040 - val_accuracy: 0.8754
Epoch 3/10
782/782 [==============================] - 35s 44ms/step - loss: 0.1319 - accuracy: 0.9530 - val_loss: 0.4069 - val_accuracy: 0.8598
Epoch 4/10
782/782 [==============================] - 35s 44ms/step - loss: 0.0813 - accuracy: 0.9726 - val_loss: 0.5010 - val_accuracy: 0.8519
Epoch 5/10
782/782 [==============================] - 34s 44ms/step - loss: 0.0526 - accuracy: 0.9825 - val_loss: 0.5380 - val_accuracy: 0.8407
Epoch 6/10
782/782 [==============================] - 34s 43ms/step - loss: 0.0408 - accuracy: 0.9864 - val_loss: 0.7301 - val_accuracy: 0.8442
Epoch 7/10
782/782 [==============================] - 34s 44ms/step - loss: 0.0268 - accuracy: 0.9917 - val_loss: 0.8390 - val_accuracy:

In [9]:
model.evaluate(x_val, y_val)

782/782 [==============================] - 12s 16ms/step - loss: 0.8571 - accuracy: 0.8323


[0.8571203947067261, 0.8322799801826477]

In [10]:
def predict_text(text):
  text = pad_sequences(text, maxlen = maxlen)
  prediction = model.predict(text)
  return prediction.argmax(axis = 1)


In [11]:
text = """It is no wonder that the film has such a high rating, it is quite literally breathtaking. What can I say that hasn't said before? Not much, it's the story, the acting, the premise, but most of all, this movie is about how it makes you feel. Sometimes you watch a film, and can't remember it days later, this film loves with you, once you've seen it, you don't forget. The ultimate story of friendship, of hope, and of life, and overcoming adversity. I understand why so many class this as the best film of all time, it isn't mine, but I get it. If you haven't seen it, or haven't seen it for some time, you need to watch it, it's amazing."""

In [12]:
predict_text(text)

ValueError: invalid literal for int() with base 10: 'I'